In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
"""
try:
  import cuml
except (ImportError, KeyError, ModuleNotFoundError):
  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
  !python rapidsai-csp-utils/colab/pip-install.py
"""

'\ntry:\n  import cuml\nexcept (ImportError, KeyError, ModuleNotFoundError):\n  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git\n  !python rapidsai-csp-utils/colab/pip-install.py\n'

In [3]:
# check if cuda is installed correctly
#!nvcc --version

In [4]:
# Mount google drive to colab and change to correct directory
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/ThinkPad/master-thesis-vt23


In [5]:
# Try to import packages, if exception is thrown install dependencies and kill runtime
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23
try:
  #import sdmetrics
  from pycaret.classification import *
  import optuna
except (ImportError, KeyError, ModuleNotFoundError):
  ## code to install dependencies
  #!pip install sdv
  #!pip install sdmetrics
  #!pip install kmodes
  #!pip install pomegranate==0.14.8   #needed for sdmetrics BNLikelihood
  !pip install pycaret==3.0.0
  !pip install optuna
  display('Stopping RUNTIME! Colaboratory will restart automatically. Please run again.')
  import os
  os.kill(os.getpid(), 9)

/content/drive/Othercomputers/ThinkPad/master-thesis-vt23


In [ ]:
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
import pandas as pd
import os
import sys
import pickle
import re
import ast

run_dataset = ['D205']
performance_df_path = f"../data/result/{run_dataset[0]}_model_performance.csv"
model_performance_df = pd.read_csv(performance_df_path)


from pycaret.classification import ClassificationExperiment
from pycaret.containers.models.classification import get_all_model_containers

from sklearn.metrics import (classification_report,
                             roc_auc_score,
                             matthews_corrcoef,
                             cohen_kappa_score)
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_validate

# Import help methods
sys.path.append('../src')
from utils import (getExperimentConfig,
                   getPicklesFromDir,
                   run_pycaret_setup,
                   translate_model_name,
                   get_synthetic_filepaths_from_original_data_id,
                   convert_and_clean_dict)

from tuning_grids import Grids

# Get global variables for the experiment
config = getExperimentConfig()
# Get folders
folders = config['folders']
# Load dataset specific settings (from the real-data)
dataset_settings = getPicklesFromDir(folders['settings_dir'])

# read performance data from Step 2
#model_performance_df = pd.read_csv(folders['model_perf_filepath'])
# Specify the metrics to sort by for choosing best model
# Choose the target metric when tuning the models
sort_by = config['clf']['tuning_param']['optimize']

for settings in dataset_settings:

    if run_dataset is not None and settings['meta']['id'] not in run_dataset:
        continue

    settings['setup_param']['fold'] = config['clf']['cv_folds']
    # update system_log name
    settings['setup_param']['system_log'] = folders['log_dir']+"Step4_SD"
    # disable saving train-test split data (to save space)
    settings['setup_param']['log_data'] = False
    target_label=settings['meta']['target']

    # Get experiment logging
    experiment_name = f"{settings['meta']['id']}-{settings['meta']['name']}"

    # load original dataset
    cols_dtype=None
    if settings['meta']['cols_dtype'] != None:
        cols_dtyped = settings['meta']['cols_dtype']

    original_data = pd.read_csv(f"{folders['real_dir']}{settings['meta']['filename']}", dtype=cols_dtype)

    X_test_df = original_data.drop(columns=[settings['meta']['target']], axis=1)
    y_test_df = original_data[settings['meta']['target']]

    logg_tags = {
        'Trained on': 'synthetic',
        'Tuned on': 'original',
    }

    # Filter the DataFrame based on the Dataset id and sort by specified column
    filtered_df = model_performance_df[model_performance_df["Dataset id"] == settings['meta']['id']]


    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(settings['meta']['id'])

    for sd_filename in synthetic_datasets:

        sd_id = os.path.splitext(sd_filename)[0]
        quality = re.findall('Q\d+', sd_id)[0]
        sd_path = folders['sd_dir']+sd_filename
        settings['setup_param']['verbose'] = False

        s = run_pycaret_setup(sd_path, settings['setup_param'], meta=settings['meta'])

        # Test the models trained on synthetic with the hyperparameters
        # recived from tuning on original data, using monte carlo stratified cv
        for _, row in filtered_df.iterrows():

            ml_model = row.model
            #get hyperparameters as dict
            hyperparameters = ast.literal_eval(row.Params)
            #buggfix: remove 'priors' from hyperparameters
            hyperparameters.pop('priors', None)

            # Add custom tags to the logg, defining dataset type, and Id
            logg_tags = {
                'Dataset id': sd_id,
                'model': ml_model,
                'Quality': quality,
                'Trained on': 'synthetic',
                'Tuned on': 'original',
                'SDG': sd_id.split("_")[0],
            }
            # Check if the setup has already been evaluated
            row_exists = ((model_performance_df['Dataset id'] == logg_tags['Dataset id']) &
                          (model_performance_df['model'] == logg_tags['model']) &
                          (model_performance_df['Tuned on'] == logg_tags['Tuned on']) &
                          (model_performance_df['Trained on'] == logg_tags['Trained on'])).any()

            if not row_exists:
                # create & tune model
                # select model based on wheter using GPU or not

                all_models = get_all_model_containers(s)
                model = all_models[ml_model].class_def()

                print(f"Tuned on: 'Original'\n{sd_id}-{translate_model_name(ml_model)}")

                tuned_model = model.set_params(**hyperparameters)
                #train the model on synthetic data
                tuned_model = s.create_model(tuned_model)
                # get validation results
                val_df = s.pull()
                val_score = {}
                val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
                val_score['val_F1'] = val_df['F1']['Mean']

                sss = StratifiedShuffleSplit(n_splits=config['clf']['mccv_iter'], train_size=settings['setup_param']['train_size'])

                metrics_list = []
                #run monte carlo stratified cross-validation using StratifiedShuffelSplit
                for _, test_index in sss.split(X_test_df, y_test_df):
                    x_test = X_test_df.iloc[test_index]
                    y_test = y_test_df.iloc[test_index]

                    x_test_transformed = s.pipeline.transform(x_test)
                    # Rearrange the column order in the same order as the train data
                    x_test_transformed = x_test_transformed[s.X_train_transformed.columns]

                    y_pred = tuned_model.predict(x_test_transformed)

                    metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
                    test_metrics = {
                        "Accuracy": metrics['accuracy'],
                        "Precision_macro": metrics['macro avg']['precision'],
                        "Recall_macro": metrics['macro avg']['recall'],
                        "F1_macro": metrics['macro avg']['f1-score'],
                        "Precision_weighted": metrics['weighted avg']['precision'],
                        "Recall_weighted": metrics['weighted avg']['recall'],
                        "F1_weighted": metrics['weighted avg']['f1-score'],
                        "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
                        "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
                    }
                    metrics_list.append(test_metrics)

                # Convert the list of dictionaries to a DataFrame
                metrics_df = pd.DataFrame(metrics_list)

                # Calculate the average of each column
                average_metrics = metrics_df.mean()

                print("Monte carlo strat-cv mean:")
                print(average_metrics)

                # save results
                performance_row = {**logg_tags, **test_metrics}
                performance_row['Params'] = tuned_model.get_params()
                model_performance_df = model_performance_df.append(performance_row, ignore_index=True)
                ########### End test hyper-param ###########

        # Start testing all models trained and tuned on synthetic
        for ml_model in config['clf']['ml_models']:
            #start log run
            logg_tags = {
                'Dataset id': sd_id,
                'model': ml_model,
                'Quality': quality,
                'Trained on': 'synthetic',
                'Tuned on': 'synthetic',
                'SDG': sd_id.split("_")[0],
            }

            # Check if the setup has already been evaluated
            row_exists = ((model_performance_df['Dataset id'] == logg_tags['Dataset id']) &
                          (model_performance_df['model'] == logg_tags['model']) &
                          (model_performance_df['Tuned on'] == logg_tags['Tuned on']) &
                          (model_performance_df['Trained on'] == logg_tags['Trained on'])).any()

            if not row_exists:

                print(f"Tuned on: 'Synthetic'\n{sd_id}-{translate_model_name(ml_model)}")

                # create & tune model
                all_models = get_all_model_containers(s)
                model = all_models[ml_model].class_def()

                # Get tuning grid
                tune_grid = Grids.get_tuning_grid(ml_model)

                # Is buggy, use default tuning by pycaret
                tuned_model = s.tune_model(model, custom_grid=tune_grid, **config['clf']['tuning_param'])

                # get validation results
                val_df = s.pull()
                val_score = {}
                val_score['val_Accuracy'] = val_df['Accuracy']['Mean']
                val_score['val_F1'] = val_df['F1']['Mean']

                sss = StratifiedShuffleSplit(n_splits=config['clf']['mccv_iter'], train_size=settings['setup_param']['train_size'])

                metrics_list = []
                #run monte carlo stratified cross-validation using StratifiedShuffelSplit
                for _, test_index in sss.split(X_test_df, y_test_df):
                    x_test = X_test_df.iloc[test_index]
                    y_test = y_test_df.iloc[test_index]

                    x_test_transformed = s.pipeline.transform(x_test)
                    # Rearrange the column order in the same order as the train data
                    x_test_transformed = x_test_transformed[s.X_train_transformed.columns]

                    y_pred = tuned_model.predict(x_test_transformed)

                    metrics =  classification_report(y_true=y_test, y_pred=y_pred, output_dict=True, digits=4)
                    test_metrics = {
                        "Accuracy": metrics['accuracy'],
                        "Precision_macro": metrics['macro avg']['precision'],
                        "Recall_macro": metrics['macro avg']['recall'],
                        "F1_macro": metrics['macro avg']['f1-score'],
                        "Precision_weighted": metrics['weighted avg']['precision'],
                        "Recall_weighted": metrics['weighted avg']['recall'],
                        "F1_weighted": metrics['weighted avg']['f1-score'],
                        "MCC": matthews_corrcoef(y_true=y_test, y_pred=y_pred),
                        "Kappa": cohen_kappa_score(y1=y_test, y2=y_pred)
                    }
                    metrics_list.append(test_metrics)

                # Convert the list of dictionaries to a DataFrame
                metrics_df = pd.DataFrame(metrics_list)

                # Calculate the average of each column
                average_metrics = metrics_df.mean()

                print(average_metrics)

                # save results and parameters
                performance_row = {**logg_tags, **average_metrics}
                performance_row['Params'] = tuned_model.get_params()
                model_performance_df = model_performance_df.append(performance_row, ignore_index=True)

        # update model performance to csv after each sd_id
        model_performance_df.to_csv(performance_df_path, index=False)

/content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
Tuned on: 'Original'
SD205Q10_0-Logistic Regression


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7127,0.4687,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.6961,0.4563,0.0000,0.0000,0.0000,-0.0324,-0.0824
2,0.7127,0.4872,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.7072,0.4378,0.0000,0.0000,0.0000,-0.0110,-0.0473
4,0.7017,0.5240,0.0000,0.0000,0.0000,-0.0110,-0.0480
5,0.7072,0.5307,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.7072,0.4869,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.7072,0.5318,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.7111,0.4845,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Monte carlo strat-cv mean:
Accuracy              0.882301
Precision_macro       0.441346
Recall_macro          0.499749
F1_macro              0.468735
Precision_weighted    0.779190
Recall_weighted       0.882301
F1_weighted           0.827546
MCC                  -0.003432
Kappa                -0.000872
dtype: float64
Tuned on: 'Synthetic'
SD205Q1000_0-Random Forest


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9061,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.9006,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9056,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Monte carlo strat-cv mean:
Accuracy              0.875442
Precision_macro       0.560528
Recall_macro          0.508954
F1_macro              0.493245
Precision_weighted    0.808623
Recall_weighted       0.875442
F1_weighted           0.830126
MCC                   0.046155
Kappa                 0.028539
dtype: float64
Tuned on: 'Synthetic'
SD205Q1000_9-Logistic Regression


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:29:43
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2023-06-16 17:29:43,995] Searching the best hyperparameters using 1808 samples...


In [ ]:
"""
%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks

import pandas as pd
import sys
import os
import re

from sdv.metadata import SingleTableMetadata

sys.path.append('../src')
from PF_metrics import compute_all_pf_measures
from utils import (getExperimentConfig,
                   getPicklesFromDir,
                   get_synthetic_filepaths_from_original_data_id)

config = getExperimentConfig()
folders = config['folders']

settings = getPicklesFromDir(folders['settings_dir'])
"""

Run pf measures for MNIST dataset

In [ ]:
"""
data_id = 'D205'
run_quality = 'Q1000'
file_path = f"../data/result/{data_id}{run_quality}_pf_measures.csv"


if os.path.exists(file_path):
    result_df = pd.read_csv(file_path)
else:
    result_df = pd.DataFrame(columns=['Dataset id'])

for dataset_setting in settings:
    #Skip specified dataset
    if data_id != dataset_setting['meta']['id']:
      continue

    original_data = pd.read_csv(folders['real_dir']+dataset_setting['meta']['filename'], dtype=dataset_setting['meta']['cols_dtype'])
    metadata = SingleTableMetadata().load_from_json(dataset_setting['meta']['meta_filepath']).to_dict()
    original_data_id = dataset_setting['meta']['id']
    synthetic_datasets = get_synthetic_filepaths_from_original_data_id(original_data_id)

    print(f"Data id: {original_data_id}")
    for sd_filename in synthetic_datasets:
        sd_id = os.path.splitext(sd_filename)[0]
        quality = re.findall('Q\d+', sd_id)[0]
        sd_path = folders['sd_dir']+sd_filename

        if run_quality != quality:
          continue

        # Check if the dataset has already been evaluated
        row_exists = ((result_df['Dataset id'] == sd_id)).any()
        if not row_exists:
            print(f"Computing PF on: {sd_id}")
            synthetic_data = pd.read_csv(sd_path)

            pf_measures = compute_all_pf_measures(original_data=original_data,
                                                  synthetic_data=synthetic_data,
                                                  dataset_meta=dataset_setting['meta'],
                                                  metadata=metadata,
                                                  SD_id=sd_id)
            pf_measures['Quality'] = quality
            pf_measures['Original dataset'] = original_data_id
            result_df = pd.concat([result_df, pf_measures], axis=0, ignore_index=True)

        # save the results
        result_df.to_csv(file_path, index=False)
"""

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step2-Model-CPU.ipynb

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step3-SDG.ipynb

In [ ]:
#%cd /content/drive/Othercomputers/ThinkPad/master-thesis-vt23/notebooks
#%run -t Step5-Measures_PF.ipynb